# Effect of repealing charitable deduction

This identifies beneficiaries of the charitable deduction by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*taxcalc version: 0.15.-  |  Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-01-19*

## Setup

### Imports

In [3]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [4]:
tc.__version__

'0.15.0'

### Load reforms

Load from Github.

In [5]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def policy_from_reform(reform):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    return pol

def create_static_policy_taxcalc_github(reform_name):
    reform = tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)
    return policy_from_reform(reform)

In [14]:
y2017_policy = create_static_policy_taxcalc_github(
    '2017_law')

In [72]:
no_charitable_reform = {2018: {'_ID_Charity_crt_all': [0]}}
y2017_no_charitable_policy = copy.deepcopy(y2017_policy)
y2017_no_charitable_policy.implement_reform(no_charitable_reform)
baseline_no_charitable_policy = tc.Policy()  # Can't combine with next step.
baseline_no_charitable_policy.implement_reform(no_charitable_reform)

### Specify `Calculator` objects for static analyses

In [16]:
recs = tc.Records.cps_constructor()

In [17]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [18]:
def static_calc_from_policy(pol, year):
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # Needs more if adding behavior.
    return calc

In [73]:
baseline_calc = static_baseline_calc(2018)
baseline_no_charitable_calc = static_calc_from_policy(no_charitable_policy, 2018)
y2017_calc = static_calc_from_policy(y2017_policy, 2018)
y2017_no_charitable_calc = static_calc_from_policy(y2017_no_charitable_policy, 2018)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Revenue-neutral UBI

Create calculators for replacing the charitable deduction with UBI under 2017 and current law.

Start by calculating the total UBI amount (same for everyone, regardless of age).

#### Change to aggregate individual income tax revenue

In [74]:
baseline_rev_change = (baseline_no_charitable_calc.weighted_total('iitax') - 
                       baseline_calc.weighted_total('iitax'))
print ('Current: ${:0.1f}B'.
       format((baseline_rev_change) / 1e9))

y2017_rev_change = (y2017_no_charitable_calc.weighted_total('iitax') - 
                    y2017_calc.weighted_total('iitax'))
print ('2017: ${:0.1f}B'.
       format((y2017_rev_change) / 1e9))

Current: $22.4B
2017: $44.8B


In [75]:
baseline_df = baseline_calc.dataframe(['s006', 'nu18', 'n1821', 'n21'])
total_population = np.sum(baseline_df['s006'] * 
                          (baseline_df['nu18'] + 
                           baseline_df['n1821'] + 
                           baseline_df['n21']))
baseline_ubi_amount = baseline_rev_change / total_population
y2017_ubi_amount = y2017_rev_change / total_population
print ('Repealing the charitable deduction could finance a UBI of ${:0.0f}'.
       format(baseline_ubi_amount) +
       ' under current law, or ${:0.0f}'.format(y2017_ubi_amount) +
       ' under 2017 law.')

Repealing the charitable deduction could finance a UBI of $68 under current law, or $136 under 2017 law.


Create `Calculator` for replacing charitable deduction with UBI.

In [77]:
def apply_ubi(existing_policy, ubi_amount):
    ubi_reform = {2018: {'_UBI_u18': [ubi_amount], 
                         '_UBI_1820': [ubi_amount], 
                         '_UBI_21': [ubi_amount],
                         '_UBI_ecrt': [1.0]}}
    result_policy = copy.deepcopy(existing_policy)
    result_policy.implement_reform(ubi_reform)
    return static_calc_from_policy(result_policy, 2018)

In [78]:
baseline_ubi_no_charitable_calc = apply_ubi(
    baseline_no_charitable_policy, baseline_ubi_amount)
y2017_ubi_no_charitable_calc = apply_ubi(
    y2017_no_charitable_policy, y2017_ubi_amount)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


Verify revenue-neutrality. 

*Use `aftertax_income` instead of `ii_tax` because the latter excludes UBI.*

In [80]:
baseline_ubi_rev_change = (baseline_ubi_no_charitable_calc.weighted_total('aftertax_income') - 
                           baseline_calc.weighted_total('aftertax_income'))
y2017_ubi_rev_change = (y2017_ubi_no_charitable_calc.weighted_total('aftertax_income') - 
                        y2017_calc.weighted_total('aftertax_income'))
print ('Revenue change from replacing charitable deduction with UBI (should be zero): ' +
       '${:0.1f}B'.format((baseline_ubi_rev_change) / 1e9) +
       ' under current law and ' +
       '${:0.1f}B'.format((y2017_ubi_rev_change) / 1e9) +
       ' under 2017 law.')

Revenue change from replacing charitable deduction with UBI (should be zero): $0.0B under current law and $0.0B under 2017 law.


### `DataFrames`

Create `DataFrames` to more flexibly analyze and remove negative income tax units, which distort the story for the bottom decile.

In [100]:
def calc_df_w_percentile(calc, remove_negatives=True):
    df = calc.dataframe(['s006', 'expanded_income', 
                         'aftertax_income']).sort_values(by=['expanded_income'])
    if remove_negatives:
        df = df.loc[df['expanded_income'] >= 0]
    df['expanded_income_percentile'] = 100 * df['s006'].cumsum() / df['s006'].sum()
    df['expanded_income_decile'] = np.ceil(df['expanded_income_percentile'] / 10).astype(int)
    # One record is getting set to 11 due to rounding.
    df.loc[df['expanded_income_decile'] == 11, 'expanded_income_decile'] = 10
    # Set as string for charting.
    df['expanded_income_decile_str'] = (
        (10 * (df['expanded_income_decile'] - 1)).map(str) + '-' + 
        (10 * df['expanded_income_decile']).map(str))
    df['total_aftertax_income'] = df['s006'] * df['aftertax_income']
    return df

In [101]:
# Use _nn convention to indicate no negatives.
baseline_df_nn = calc_df_w_percentile(baseline_calc)
y2017_df_nn = calc_df_w_percentile(y2017_calc)
baseline_no_charitable_df_nn = calc_df_w_percentile(baseline_no_charitable_calc)
y2017_no_charitable_df_nn = calc_df_w_percentile(y2017_no_charitable_calc)
baseline_ubi_df_nn = calc_df_w_percentile(baseline_ubi_no_charitable_calc)
y2017_ubi_df_nn = calc_df_w_percentile(y2017_ubi_no_charitable_calc)

## Analysis

### Difference tables by income decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [91]:
baseline_diff_table = baseline_calc.difference_table(baseline_no_charitable_calc, tax_to_diff='iitax')
y2017_diff_table = y2017_calc.difference_table(y2017_no_charitable_calc, tax_to_diff='iitax')
baseline_ubi_diff_table = baseline_calc.difference_table(baseline_ubi_no_charitable_calc, tax_to_diff='iitax')
y2017_ubi_diff_table = y2017_calc.difference_table(y2017_ubi_no_charitable_calc, tax_to_diff='iitax')

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


In [92]:
baseline_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"62,591.29",0.00,-0.00,0.00
1,"16,988,285.70",0.00,0.00,"5,207.10",0.03,0.01,"222,332.56",0.00,0.00,-0.00
2,"16,990,004.05",0.00,0.00,"133,144.85",0.78,1.41,"23,923,356.00",0.11,0.01,-0.01
3,"16,989,656.05",0.00,0.00,"332,876.29",1.96,5.39,"91,611,714.09",0.41,0.02,-0.02
4,"16,989,954.66",0.00,0.00,"615,923.46",3.63,13.49,"229,159,421.60",1.02,0.04,-0.04
5,"16,989,240.80",0.00,0.00,"960,374.81",5.65,29.21,"496,188,132.72",2.21,0.07,-0.07
6,"16,989,686.38",0.00,0.00,"1,536,497.48",9.04,76.15,"1,293,752,014.36",5.77,0.15,-0.15
7,"16,989,484.52",0.00,0.00,"2,781,233.05",16.37,128.86,"2,189,182,040.94",9.76,0.19,-0.19
8,"16,989,613.44",0.00,0.00,"5,451,774.32",32.09,308.49,"5,241,064,357.63",23.36,0.32,-0.32
9,"16,989,497.48",0.00,0.00,"7,128,114.04",41.96,757.63,"12,871,824,279.86",57.37,0.35,-0.35


In [93]:
y2017_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"43,611.20",0.00,-0.00,0.00
1,"16,988,285.70",0.00,0.00,"50,187.09",0.30,0.28,"4,760,463.72",0.01,0.00,-0.00
2,"16,990,004.05",0.00,0.00,"691,813.67",4.07,5.15,"87,448,642.66",0.20,0.03,-0.03
3,"16,989,656.05",0.00,0.00,"1,903,067.48",11.20,27.14,"461,137,528.32",1.03,0.11,-0.11
4,"16,989,954.66",0.00,0.00,"2,738,159.87",16.12,64.25,"1,091,628,802.00",2.44,0.21,-0.21
5,"16,989,240.80",0.00,0.00,"3,682,693.42",21.68,119.80,"2,035,392,083.49",4.55,0.31,-0.31
6,"16,989,686.38",0.00,0.00,"4,935,193.47",29.05,254.36,"4,321,435,690.84",9.66,0.50,-0.50
7,"16,989,484.52",0.00,0.00,"5,809,046.03",34.19,399.90,"6,794,116,838.92",15.18,0.59,-0.59
8,"16,989,613.44",0.00,0.00,"7,050,328.08",41.50,626.33,"10,641,103,380.35",23.78,0.65,-0.65
9,"16,989,497.48",0.00,0.00,"8,284,178.18",48.76,"1,136.78","19,313,295,549.07",43.16,0.54,-0.54


UBI diff tables. % after-tax income for bottom decile is skewed due to that group having negative after-tax income in the baseline.

In [95]:
baseline_ubi_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"62,591.29",0.00,-0.00,-34.25
1,"16,988,285.70",0.00,0.00,"5,207.10",0.03,0.01,"222,332.56",0.00,0.00,0.98
2,"16,990,004.05",0.00,0.00,"133,144.85",0.78,1.41,"23,923,356.00",0.11,0.01,0.59
3,"16,989,656.05",0.00,0.00,"332,876.29",1.96,5.39,"91,611,714.09",0.41,0.02,0.46
4,"16,989,954.66",0.00,0.00,"615,923.46",3.63,13.49,"229,159,421.60",1.02,0.04,0.36
5,"16,989,240.80",0.00,0.00,"960,374.81",5.65,29.21,"496,188,132.72",2.21,0.07,0.26
6,"16,989,686.38",0.00,0.00,"1,536,497.48",9.04,76.15,"1,293,752,014.36",5.77,0.15,0.12
7,"16,989,484.52",0.00,0.00,"2,781,233.05",16.37,128.86,"2,189,182,040.94",9.76,0.19,0.03
8,"16,989,613.44",0.00,0.00,"5,451,774.32",32.09,308.49,"5,241,064,357.63",23.36,0.32,-0.14
9,"16,989,497.48",0.00,0.00,"7,128,114.04",41.96,757.63,"12,871,824,279.86",57.37,0.35,-0.27


In [96]:
y2017_ubi_diff_table

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"43,611.20",0.00,-0.00,-67.13
1,"16,988,285.70",0.00,0.00,"50,187.09",0.30,0.28,"4,760,463.72",0.01,0.00,1.95
2,"16,990,004.05",0.00,0.00,"691,813.67",4.07,5.15,"87,448,642.66",0.20,0.03,1.18
3,"16,989,656.05",0.00,0.00,"1,903,067.48",11.20,27.14,"461,137,528.32",1.03,0.11,0.86
4,"16,989,954.66",0.00,0.00,"2,738,159.87",16.12,64.25,"1,091,628,802.00",2.44,0.21,0.60
5,"16,989,240.80",0.00,0.00,"3,682,693.42",21.68,119.80,"2,035,392,083.49",4.55,0.31,0.36
6,"16,989,686.38",0.00,0.00,"4,935,193.47",29.05,254.36,"4,321,435,690.84",9.66,0.50,0.04
7,"16,989,484.52",0.00,0.00,"5,809,046.03",34.19,399.90,"6,794,116,838.92",15.18,0.59,-0.14
8,"16,989,613.44",0.00,0.00,"7,050,328.08",41.50,626.33,"10,641,103,380.35",23.78,0.65,-0.30
9,"16,989,497.48",0.00,0.00,"8,284,178.18",48.76,"1,136.78","19,313,295,549.07",43.16,0.54,-0.36


### % change to after-tax income

Focus on % change to after-tax income. Build new `DataFrame`s to address portion of bottom decile with negative income.

In [103]:
def decile_after_tax_summary(calc):
    return calc.groupby('expanded_income_decile_str', 
                        as_index=False)['total_aftertax_income'].sum()

In [105]:
# Use _s to indicate summary.
baseline_df_nn_s = decile_after_tax_summary(baseline_df_nn)
baseline_df_nn_s = decile_after_tax_summary(y2017_df_nn)
baseline_no_charitable_df_nn_s = decile_after_tax_summary(baseline_no_charitable_df_nn)
y2017_no_charitable_df_nn_s = decile_after_tax_summary(y2017_no_charitable_df_nn)
baseline_ubi_df_nn_s = decile_after_tax_summary(baseline_ubi_df_nn)
y2017_ubi_df_nn_s = decile_after_tax_summary(y2017_ubi_df_nn)

In [107]:
def decile_comparison(base, comparison):
    decile_comparison = pd.merge(base, comparison, on='expanded_income_decile_str')
    decile_comparison['pct_change'] = (
        decile_comparison['total_aftertax_income_y'] / 
        decile_comparison['total_aftertax_income_x']) - 1
    return decile_comparison

In [110]:
baseline_ubi_diff = decile_comparison(baseline_df_nn_s, baseline_ubi_df_nn_s)
baseline_ubi_diff

,expanded_income_decile_str,total_aftertax_income_x,total_aftertax_income_y,pct_change
0,0-10,"2,464,370,058,582.7%","2,647,550,910,920.1%",7.4%
1,10-20,"17,009,079,034,587.7%","17,217,263,602,075.5%",1.2%
2,20-30,"28,833,809,338,307.5%","29,144,460,822,359.9%",1.1%
3,30-40,"40,297,883,081,050.5%","40,855,759,746,341.6%",1.4%
4,40-50,"52,057,518,326,131.9%","52,953,462,612,647.7%",1.7%
5,50-60,"66,590,307,483,705.8%","67,765,709,978,824.9%",1.8%
6,60-70,"86,138,773,633,968.4%","87,418,802,329,959.8%",1.5%
7,70-80,"115,304,052,562,918.0%","116,968,797,481,320.2%",1.4%
8,80-90,"162,837,394,764,916.2%","164,824,583,676,127.0%",1.2%
9,90-100,"359,667,306,386,032.0%","365,993,094,176,110.0%",1.8%


In [97]:
pd.options.display.float_format = '{:,.1%}'.format
pd.DataFrame({'baseline': baseline_diff_table['pc_aftertaxinc'],
              'y2017': y2017_diff_table['pc_aftertaxinc']})

,baseline,y2017
0,0.1%,0.1%
1,-0.0%,-0.3%
2,-0.8%,-3.0%
3,-2.3%,-11.5%
4,-4.3%,-21.0%
5,-7.3%,-30.6%
6,-14.8%,-50.2%
7,-18.7%,-58.9%
8,-31.7%,-65.3%
9,-35.1%,-53.7%


Same thing for UBI.

In [98]:
pd.DataFrame({'baseline': baseline_ubi_diff_table['pc_aftertaxinc'],
              'y2017': y2017_ubi_diff_table['pc_aftertaxinc']})

,baseline,y2017
0,"-3,425.4%","-6,713.2%"
1,97.6%,195.1%
2,59.5%,117.9%
3,46.0%,85.8%
4,35.7%,60.0%
5,25.6%,36.1%
6,12.0%,4.0%
7,3.3%,-14.3%
8,-14.1%,-29.7%
9,-26.5%,-36.3%


Graphed in Google Sheets [here](https://docs.google.com/spreadsheets/d/17ozKKuWUzgI4yn1OwqshmPBVWVklQ1GlpTAUepuwsF8/edit?usp=sharing).

*TODO: Make a Seaborn or Bokeh plot here instead.*

### After-tax income held by top decile

Ignore errors ([issue](https://github.com/open-source-economics/Tax-Calculator/issues/1799)).

In [87]:
baseline_dist, baseline_no_charitable_dist = baseline_calc.distribution_tables(baseline_no_charitable_calc)
y2017_dist, y2017_no_charitable_dist = y2017_calc.distribution_tables(y2017_no_charitable_calc)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


In [88]:
def after_tax_income_top10pct(dist):
    return dist.aftertax_income[9] / dist.aftertax_income[10]

In [89]:
after_tax_10p_share = pd.DataFrame({
    'baseline': [after_tax_income_top10pct(y2017_dist),
                 after_tax_income_top10pct(baseline_dist)],
    'no_charitable': [after_tax_income_top10pct(y2017_no_charitable_dist),
                      after_tax_income_top10pct(baseline_no_charitable_dist)]
}, index=pd.Series([2017, 2018], name='tax_year'))
after_tax_10p_share['diff'] = (after_tax_10p_share['no_charitable'] -
                               after_tax_10p_share['baseline'])

In [90]:
pd.options.display.float_format = '{:,.3%}'.format
print('Share of after-tax income from top decile')
after_tax_10p_share

Share of after-tax income from top decile


,baseline,no_charitable,diff
tax_year,,,
2017,38.773%,38.751%,-0.021%
2018,38.946%,38.902%,-0.044%


## Revenue-neutral UBI

Calculate UBI amount, consistent across age groups.

In [58]:
tcja_ubi_diff = tcja_calc.difference_table(tcja_ubi_no_charitable_calc, tax_to_diff='iitax')
tcja_ubi_diff

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"16,989,416.58",0.00,0.00,500.80,0.00,0.00,"62,591.29",0.00,-0.00,-34.24
1,"16,988,285.70",0.00,0.00,"5,207.10",0.03,0.01,"222,332.56",0.00,0.00,0.98
2,"16,990,004.05",0.00,0.00,"133,144.85",0.78,1.41,"23,923,356.00",0.11,0.01,0.59
3,"16,989,656.05",0.00,0.00,"332,876.29",1.96,5.39,"91,611,714.09",0.41,0.02,0.46
4,"16,989,954.66",0.00,0.00,"615,923.46",3.63,13.49,"229,159,421.60",1.02,0.04,0.36
5,"16,989,240.80",0.00,0.00,"960,374.81",5.65,29.21,"496,188,132.72",2.21,0.07,0.26
6,"16,989,686.38",0.00,0.00,"1,536,497.48",9.04,76.15,"1,293,752,014.36",5.77,0.15,0.12
7,"16,989,484.52",0.00,0.00,"2,781,233.05",16.37,128.86,"2,189,182,040.94",9.76,0.19,0.03
8,"16,989,613.44",0.00,0.00,"5,451,774.32",32.09,308.49,"5,241,064,357.63",23.37,0.32,-0.14
9,"16,989,497.48",0.00,0.00,"7,132,252.57",41.98,757.16,"12,863,684,595.64",57.35,0.35,-0.26


In [66]:
tcja_dist, tcja_ubi_no_charitable_dist = tcja_calc.distribution_tables(tcja_ubi_no_charitable_calc)
tcja_ubi_no_charitable_dist

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


,num_returns_AMT,num_returns_ItemDed,s006,num_returns_StandardDed,refund,taxbc,c04600,c00100,iitax,aftertax_income,...,expanded_income,standard,payrolltax,c04470,c05800,combined,c04800,c62100,othertaxes,c09600
0,0,"5,192","16,989,417","9,521,653","1,749,494,526","12,340,505",0,"-10,193,589,279","-1,743,700,380","-3,424,903,775",...,"-2,170,926,654","195,446,451,518","2,997,677,501","35,165,350","12,340,505","1,253,977,121","414,186,102","-10,227,372,044",0,0
1,"9,254","21,495","16,988,286","11,740,583","13,194,877,004","52,538,473",0,"87,166,220,742","-13,163,904,653","170,840,148,316",...,"168,939,519,698","240,015,302,635","11,263,276,035","308,852,704","57,532,032","-1,900,628,617","1,451,827,055","86,950,972,101",0,"4,993,559"
2,"27,895","182,872","16,990,004","13,234,813","18,438,107,191","2,419,042,833",0,"155,153,469,210","-16,746,369,568","290,950,329,761",...,"294,743,794,341","253,803,269,259","20,539,834,149","3,729,227,185","2,435,441,581","3,793,464,581","25,994,093,905","152,120,887,043",0,"16,398,748"
3,"30,459","231,175","16,989,656","15,510,181","22,932,260,184","9,104,500,674",0,"289,412,336,599","-16,580,368,842","408,202,503,706",...,"431,159,348,489","273,611,457,022","39,537,213,625","5,139,143,545","9,140,332,164","22,956,844,783","91,973,127,168","285,244,855,011",0,"35,831,490"
4,"40,800","357,070","16,989,955","16,236,349","19,042,074,111","20,726,702,795",0,"446,608,860,087","-4,153,035,367","529,407,744,034",...,"586,577,563,738","285,521,293,907","61,322,855,071","8,483,292,194","20,800,173,606","57,169,819,704","200,282,438,496","439,810,177,891",0,"73,470,811"
5,"34,850","627,105","16,989,241","16,264,181","10,509,569,143","37,987,374,635",0,"644,501,853,799","17,881,738,892","677,219,384,045",...,"783,137,896,218","292,045,608,677","88,036,773,282","14,253,051,092","38,096,616,054","105,918,512,174","359,780,411,948","633,686,727,185",0,"109,241,420"
6,"34,072","1,614,129","16,989,686","15,366,221","3,170,636,117","70,114,227,737",0,"930,309,462,377","53,025,001,430","874,255,610,283",...,"1,053,269,948,558","290,234,097,836","125,989,336,844","33,413,820,798","70,206,780,060","179,014,338,274","607,287,527,469","906,330,554,474",0,"92,552,323"
7,"33,051","2,491,415","16,989,485","14,498,070","588,535,519","124,661,959,526",0,"1,337,000,323,611","105,581,118,436","1,170,018,185,967",...,"1,458,309,316,741","300,410,290,263","182,710,012,337","50,513,880,553","124,779,806,439","288,291,130,773","980,583,099,052","1,303,874,440,503",0,"117,846,913"
8,"32,184","3,080,464","16,989,613","13,909,149","47,901,870","218,581,616,524",0,"1,953,272,033,497","195,291,249,454","1,649,158,170,294",...,"2,113,903,052,384","310,480,926,632","269,453,632,637","72,661,936,271","218,679,194,977","464,744,882,090","1,560,410,846,977","1,908,831,549,356",0,"97,578,454"
9,"324,722","7,543,810","16,989,497","9,445,688","114,766","852,786,686,975",0,"4,684,346,932,012","841,481,098,717","3,665,600,969,788",...,"4,938,794,554,336","216,496,777,897","431,712,485,831","213,781,487,184","855,571,625,741","1,273,193,584,549","4,220,090,942,211","4,546,116,002,372","9,355,185,401","2,784,938,766"


In [67]:
tcja_dist

,num_returns_AMT,num_returns_ItemDed,s006,num_returns_StandardDed,refund,taxbc,c04600,c00100,iitax,aftertax_income,...,expanded_income,standard,payrolltax,c04470,c05800,combined,c04800,c62100,othertaxes,c09600
0,0,"5,767","16,989,417","9,521,078","1,749,494,526","12,273,093",0,"-10,193,589,279","-1,743,762,971","-5,208,335,561",...,"-3,954,421,032","195,445,667,321","2,997,677,501","36,623,671","12,273,093","1,253,914,530","413,511,978","-10,228,629,095",0,0
1,"8,196","28,814","16,988,286","11,733,264","13,194,877,004","52,371,797",0,"87,166,220,742","-13,164,126,985","169,188,714,986",...,"167,287,864,037","239,932,658,612","11,263,276,035","437,764,103","56,825,001","-1,900,850,950","1,452,285,900","86,846,774,444",0,"4,453,204"
2,"27,533","303,946","16,990,004","13,113,739","18,438,212,508","2,391,176,381",0,"155,153,469,210","-16,770,292,924","289,230,111,626",...,"292,999,652,851","252,315,232,620","20,539,834,149","5,797,741,422","2,407,402,849","3,769,541,225","25,695,916,485","150,438,736,054",0,"16,226,469"
3,"28,055","551,283","16,989,656","15,190,074","22,939,722,229","9,004,826,070",0,"289,412,336,599","-16,671,980,556","406,332,142,171",...,"429,197,375,240","269,589,777,528","39,537,213,625","10,401,526,637","9,037,688,848","22,865,233,069","91,015,452,628","281,031,797,290",0,"32,862,777"
4,"38,282","911,873","16,989,955","15,681,546","19,061,509,219","20,488,841,965",0,"446,608,860,087","-4,382,194,788","527,525,374,783",...,"584,466,035,065","278,455,596,297","61,322,855,071","17,957,208,215","20,559,474,866","56,940,660,283","198,171,371,421","432,457,528,125",0,"70,632,902"
5,"32,672","1,401,742","16,989,241","15,489,544","10,541,113,265","37,499,167,795",0,"644,501,853,799","17,385,550,759","675,489,794,338",...,"780,912,118,379","281,563,394,615","88,036,773,282","29,209,313,844","37,590,988,846","105,422,324,041","355,544,023,377","622,319,589,528",0,"91,821,051"
6,"33,652","2,558,704","16,989,686","14,421,646","3,210,451,694","68,816,962,108",0,"930,309,462,377","51,731,249,416","873,209,578,287",...,"1,050,930,164,547","275,024,148,267","125,989,336,844","56,487,059,476","68,906,795,906","177,720,586,260","599,674,744,007","888,647,643,142",0,"89,833,799"
7,"32,954","4,263,398","16,989,485","12,726,086","635,957,139","122,473,220,968",0,"1,337,000,323,611","103,391,936,395","1,169,630,979,294",...,"1,455,732,928,026","262,439,767,176","182,710,012,337","100,434,424,036","122,586,698,491","286,101,948,733","968,827,409,181","1,266,410,406,829",0,"113,477,523"
8,"31,042","7,098,783","16,989,613","9,890,831","56,283,879","213,340,778,776",0,"1,953,272,033,497","190,050,185,096","1,651,490,703,111",...,"2,110,994,520,844","221,466,422,843","269,453,632,637","186,522,521,713","213,435,476,195","459,503,817,733","1,535,606,125,012","1,828,507,545,078",0,"94,697,418"
9,"314,061","10,634,624","16,989,497","6,354,874","247,192","839,981,806,900",0,"4,684,346,932,012","828,617,414,122","3,675,339,079,382",...,"4,935,668,979,335","146,948,117,820","431,712,485,831","333,522,682,955","842,707,941,145","1,260,329,899,953","4,169,899,948,888","4,455,240,500,520","9,355,185,401","2,726,134,245"


In [61]:
fig = tcja_calc.decile_graph(tcja_ubi_no_charitable_calc)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


In [64]:
output_notebook()

Loading BokehJS ...

In [65]:
show(fig)